In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Function Calling은 GPT-3, GPT-4에서만 지원하는 기능으로 모델이 특정 함수를 호출하도록 할 수 있음
# 이 기능을 통해 모델이 특정 함수를 호출하도록 하거나, 원하는 특정한 형식의 output을 갖도록 강제할 수 있음


"""
    예를 들어서 다음과 같은 코드가 있을 때,

    llm  = ChatOpenAI(
        temperature=0.1
    )

    prompt = PromptTemplate.from_template("How is the weather in {city}")

    chain = prompt | llm

    chain.invoke({
        "city":"rome"
    })
    
    위 내용을 실행시키면 다음과 같은 답변이 돌아옴 

    AIMessage(content="I'm sorry, I cannot provide real-time weather updates as I am an AI language model and do not have access to current data. However, you can check the weather in Rome by using a search engine or a weather website or app.")

    그런데 실시간 데이터를 가져올 수 있는 함수가 있다면?
    AI가 그걸 호출해서 데이터를 가져오도록 하면 되지 않을까?
    이럴때 Function Calling을 사용가능 아래 참고
"""


def get_weather(lng, lat):
    print("call an api...")

function = {
    "name" : "get_weather",
    "description":"function that takes longitude and latitude to find the weather of a place",
    "parameters" : {
        "type" : "object",
        "properties":{
            "lng":{
                "type":"string",
                "description":"The longitude coordinate"
            },
            "lat":{
                "type":"string",
                "description":"The latitude coordinate"
            }
        }
    },
    "required":["lng","lat"]
} # 함수의 형태와 필요한 데이터를 설명하는 schema 작성


llm  = ChatOpenAI(
    temperature=0.1
).bind(
    # function_call={"name" :"get_weather"},# 모델이 함수를 사용하도록 강제하고 싶은 경우 함수명을 정확히 입력
    function_call="auto", # AI가 필요에 따라 선택하여 사용하도록 하고 싶은 경우
    functions=[function]
) # bind함수를 이용하여 function_call 속성을 지정가능

prompt = PromptTemplate.from_template("How is the weather in {city}")

chain = prompt | llm

response = chain.invoke({
    "city":"rome"
})

response = response.additional_kwargs["function_call"]["arguments"]

Authentication failed for https://api.smith.langchain.com/runs. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs', '{"detail":"Invalid auth"}')
Authentication failed for https://api.smith.langchain.com/runs/dd08bba9-7bb8-4835-9151-e2304e083222. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/dd08bba9-7bb8-4835-9151-e2304e083222', '{"detail":"Invalid auth"}')


In [3]:
import json

r = json.loads(response) # json문자열을 파이썬 객체로 변환

get_weather(r['lng'], r['lat'])

call an api...


In [4]:
# Function Call기능을 이용하면 실제로 함수가 존재하지 않더라도, 가짜 함수와 함께 함수의 입력 파라미터를 명시함으로써 llm의 출력이 항상 명시한 파라미터 형태로 나오도록 강제할 수 있음

# 이렇게 하면 llm을 두번 호출할 필요없음
# (기존에는 답변을 생성하기 위해 1번 호출하고, format을 위해 다시 llm을 호출했었음)
# 그러나 이 기능은 gpt-3, gpt-4에서만 사용 가능함

function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz", # llm이 항상 create_quiz (llm의 출력형태를 고정시키기 위해 작성 한 실제로는 존재하지 않는 가짜 함수)를 호출하도록 하기 위해 name 명시
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{\n  "questions": [\n    {\n      "question": "What year was Rome founded?",\n      "answers": [\n        {\n          "answer": "753 BC",\n          "correct": true\n        },\n        {\n          "answer": "476 AD",\n          "correct": false\n        },\n        {\n          "answer": "1492 AD",\n          "correct": false\n        },\n        {\n          "answer": "44 BC",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "Who was the first emperor of Rome?",\n      "answers": [\n        {\n          "answer": "Julius Caesar",\n          "correct": false\n        },\n        {\n          "answer": "Augustus",\n          "correct": true\n        },\n        {\n          "answer": "Nero",\n          "correct": false\n        },\n        {\n          "answer": "Constantine",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "What was the official language of the Roman Empire?",\n      "answers": [\n        {\n       

In [5]:
for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What year was Rome founded?', 'answers': [{'answer': '753 BC', 'correct': True}, {'answer': '476 AD', 'correct': False}, {'answer': '1492 AD', 'correct': False}, {'answer': '44 BC', 'correct': False}]}
{'question': 'Who was the first emperor of Rome?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}, {'answer': 'Constantine', 'correct': False}]}
{'question': 'What was the official language of the Roman Empire?', 'answers': [{'answer': 'Latin', 'correct': True}, {'answer': 'Greek', 'correct': False}, {'answer': 'Italian', 'correct': False}, {'answer': 'Spanish', 'correct': False}]}
{'question': 'Who was the legendary founder of Rome?', 'answers': [{'answer': 'Romulus', 'correct': True}, {'answer': 'Remus', 'correct': False}, {'answer': 'Caesar', 'correct': False}, {'answer': 'Hannibal', 'correct': False}]}
{'question': 'What was the name of the Roman gladiatorial arena?', 'answers': [{